In [ ]:
# ✅ Import thư viện
import pandas as pd
from sqlalchemy import create_engine
from time import time

# ✅ Khai báo thông tin kết nối PostgreSQL
user = "root"
password = "root"
host = "localhost"     # hoặc 'pgdatabase' nếu chạy trong Docker
port = 5432
db = "ny_taxi"
table_name = "yellow_taxi_data"

# ✅ Đường dẫn tới file CSV hoặc Parquet
csv_file = "E:/yellow_tripdata_2024-01.csv"
# parquet_file = "E:/yellow_tripdata_2024-01.parquet"

# ✅ Tạo engine kết nối
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

# ✅ Đọc file theo từng chunk (tránh out of memory)
df_iter = pd.read_csv(csv_file, iterator=True, chunksize=100000)

for df in df_iter:
    t_start = time()

    # ✅ Chuyển datetime nếu có cột thời gian
    if 'tpep_pickup_datetime' in df.columns:
        df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    if 'tpep_dropoff_datetime' in df.columns:
        df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

    # ✅ Ghi vào PostgreSQL
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

    t_end = time()
    print(f"✔️ Inserted a chunk in {t_end - t_start:.2f} seconds")

engine.dispose()


In [ ]:
SELECT COUNT(*) FROM yellow_taxi_data;